<a href="https://colab.research.google.com/github/khirotaka/Swift4TF/blob/master/Colab/Swift%20for%20TensorFlow%20for%20using%20PyTorch%20users%20No1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch使いの為の Swift for TensorFlow

In [0]:
import TensorFlow

In [2]:
TensorFlow.randomSeedForTensorFlow(using: TensorFlowSeed(graph: 0, op: 0))    // Seed固定

var x: Tensor<Float> = _Raw.empty(shape: [5, 3])    // TensorFlowの低レベルAPIにアクセスするには _Raw を使う。(Raw は非推奨になった)
print(x)

[[0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0]]


Swift for TensorFlowには `ShapedArray` というものがあり、`numpy.ndarray` に近いものになっている。

In [6]:
var xx: ShapedArray<Float> = ShapedArray<Float>(shape: [5, 5], repeating: 0.0)
print("Shape of xx \(xx.shape)")
print(xx)

Shape of xx [5, 5]
[[0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0]]


Swift for TensorFlowで、
```python
torch.randn(5, 3)
```
を行いたいのなら、次のようにすればいい。

In [7]:
var x: Tensor<Float> = Tensor<Float>(randomNormal: [5, 3])
print(x)

[[  0.4156954, -0.30111942,  -1.3234228],
 [ -1.1048259, -0.22339313, 0.082799174],
 [ 0.50594443,  -0.3716898,  0.35465118],
 [-0.24135874,  -0.5222881,  -0.2871418],
 [  0.1728122,  -0.7446935,   0.6472354]]


```python
torch.zeros(5, 3)
```
や
```python
torch.ones(5, 3)
```
も以下のように書ける。

In [11]:
var x = Tensor<Float>(ones: [5, 3])
print(x)

print("----------")    // swift で "-"*50 が使えないのホント不便

var xx = Tensor<UInt8>(zeros: [5, 3])
print(xx)

[[1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0]]
----------
[[0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0]]


`zeros_like` も `ones_like` もある。

In [18]:
var tmp = ShapedArray<Float>(repeating: 0.0, shape: [5, 5])

var x = Tensor<Float>(onesLike: Tensor<Float>(tmp))
print(x)

print("----------")

var xx = Tensor<Float>(zerosLike: Tensor<Float>(tmp))
print(xx)

[[1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0]]
----------
[[0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0]]


当然、
```python
torch.tensor([5.5, 3.0])
```
のような書き方も可能

In [19]:
var x = Tensor<Float>([5.5, 3])
print(x)

[5.5, 3.0]


先ほどから出ている正規分布からサンプリングした乱数以外にも

In [20]:
var y = Tensor<Float>(randomNormal: [5, 3])

print(y)

[[  0.2105479,  -1.3495227, -0.16156627],
 [ -1.3012795,    1.319071,   2.0498898],
 [ -0.9137847, -0.47593895,  0.25041267],
 [  1.3437054,  0.74800974,   1.3092104],
 [ -0.6983921,  0.47109076,   0.6180191]]


## Using GPU

```swift
var x = withDevice(.gpu, 0) {
    Tensor<Float>([1, 2, 3])
}    // これでGPUに載せる事が出来る。

```

In [0]:
import Foundation    // Dataを使う為

In [24]:
let start = Date()

var x = TensorFlow.withDevice(.gpu, 0) {
    Tensor<Float>(randomNormal: [5000, 6000])
}

var y = TensorFlow.withDevice(.gpu, 0) {
    matmul(x, x.transposed())
}

let end = Date().timeIntervalSince(start)
print(end)

0.689223051071167


In [25]:
let start = Date()

var x = TensorFlow.withDevice(.cpu, 0) {
    Tensor<Float>(randomNormal: [5000, 6000])
}

var y = TensorFlow.withDevice(.cpu, 0) {
    matmul(x, x.transposed())
}

let end = Date().timeIntervalSince(start)
print(end)

3.977492928504944


## Numpyとの互換性

### Numpy -> TensorFlow on Swift

まず、Pythonと site-packages にインストールされているPythonライブラリを使う。

In [0]:
import PythonKit

let np = Python.import("numpy")

In [38]:
Python.version    // Python のバージョン

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]


```python
let np = Python.import("numpy")
```

で`numpy` が使えるようになります。
あとは、いつもPythonで書いてるように関数やオブジェクトにアクセスできます。

In [39]:
var np_a = np.random.randn(512, 32)
print(np_a.shape)    // Numpy オブジェクト

(512, 32)


In [0]:
var swift_a = Tensor<Double>(numpy: np_a)!    // Numpyを元に Tensor を作成。オプショナル型なので注意


In [50]:
print(swift_a.shape)

[512, 32]


### Swift -> Numpy

In [51]:
var new_ndarray = swift_a.makeNumpyArray()    // PyTorch の numpy() と同じノリ

print(new_ndarray.shape)

(512, 32)


Pythonに内蔵されている関数 (例えば、 `type` を使いたいのなら、以下のようにすれば行ける)

In [53]:
Python.type(new_ndarray)

<class 'numpy.ndarray'>
